
# S.S. Lootah Real Estate — Exploratory Data Analysis (EDA)

**Objective:** Provide descriptive analytics summarizing *revenue, occupancy, and expired contracts per building*, plus basic data cleaning and structure checks to prepare for downstream modeling.

**Datasets (CSV):**
- `Total Contracts-Table 1.csv`
- `Unit Master-Table 1.csv`
- `Tenancy contracts with previous rents.csv`

**Key variables to prioritize:** `Amount (Revenue)`, `Contract Status`, `Property Name`, `Contract Start Date`, `Contract End Date`, `Previous Tenancy Rent`, `Building Name`, `Unit Type`, `Rental Status`.

---

**Notebook Map**
1. Imports, Settings, and Helper Functions  
2. Data Loading and Initial Inspection  
3. Cleaning & Standardization  
4. Feature Engineering (derived fields)  
5. Descriptive Statistics & Quality Checks  
6. Visualizations  
   - Distributions (Revenue)  
   - Revenue vs. Previous Rent (Per Property/Building)  
   - Occupancy Rate (Per Building)  
   - Expired Contracts (Per Building)  
7. Findings & Next Steps per section  
8. Executive Summary (non-technical)



## 1) Imports, Settings, and Helper Functions

**What & Why:**  
- Import core libraries (`pandas`, `numpy`, `matplotlib`, `seaborn`) for data processing and visualization.  
- Set global display and plotting defaults to keep outputs consistent.  
- Define small helper utilities for safe parsing, cleaning strings, and plotting.


In [ ]:

# --- Standard imports
import os
import re
import math
from pathlib import Path
from datetime import datetime, date

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

# Plotting defaults
plt.rcParams['figure.figsize'] = (10, 6)
plt.rcParams['axes.grid'] = True
sns.set_theme(context='notebook', style='whitegrid')

# Paths (adjust if running elsewhere)
DATA_DIR = Path('/mnt/data')  # update if needed
FILE_TOTAL_CONTRACTS = DATA_DIR / 'Total Contracts-Table 1.csv'
FILE_UNIT_MASTER = DATA_DIR / 'Unit Master-Table 1.csv'
FILE_PREV_RENTS = DATA_DIR / 'Tenancy contracts with previous rents.csv'

# Today's date in Dubai timezone context (approx; tz not applied in arithmetic here)
TODAY = pd.Timestamp('2025-10-31')

def to_num(x):
    """Safely convert strings like '1,234.50', 'AED 5,000', or None to float."""
    if pd.isna(x):
        return np.nan
    s = str(x)
    s = re.sub(r'[^0-9.\-]', '', s)  # keep digits, decimal point, minus
    try:
        return float(s) if s else np.nan
    except:
        return np.nan

def to_dt(x):
    """Robust datetime parser; returns NaT if parsing fails."""
    return pd.to_datetime(x, errors='coerce', dayfirst=False, infer_datetime_format=True)

def std_colnames(df):
    """Standardize column names: strip, lower, replace spaces and special chars with underscores."""
    df = df.copy()
    df.columns = (
        df.columns.str.strip()
                  .str.replace('\n', ' ', regex=False)
                  .str.replace(r'[^0-9a-zA-Z]+', '_', regex=True)
                  .str.strip('_')
                  .str.lower()
    )
    return df

def safe_read_csv(path, **kwargs):
    """Read CSV with common encodings and separators fallback."""
    encodings = ['utf-8', 'utf-8-sig', 'cp1252', 'iso-8859-1']
    seps = [',', ';', '|', '\t']
    last_err = None
    for enc in encodings:
        for sep in seps:
            try:
                return pd.read_csv(path, encoding=enc, sep=sep, **kwargs)
            except Exception as e:
                last_err = e
                continue
    raise last_err

def topn(series, n=10):
    """Return top-n value counts as DataFrame with percentages."""
    vc = series.value_counts(dropna=False)
    pct = (vc / vc.sum() * 100).round(2)
    return pd.DataFrame({'count': vc, 'percent': pct}).head(n)



## 2) Data Loading and Initial Inspection

**What & Why:**  
- Load the three datasets robustly with fallback handling.  
- Preserve raw copies; create standardized versions with consistent column names.  
- Quick shape, column overview, and sample rows to understand structure.


In [ ]:

# Load
raw_contracts = safe_read_csv(FILE_TOTAL_CONTRACTS)
raw_units = safe_read_csv(FILE_UNIT_MASTER)
raw_prev_rents = safe_read_csv(FILE_PREV_RENTS)

# Keep raw backups
contracts = std_colnames(raw_contracts)
units = std_colnames(raw_units)
prev_rents = std_colnames(raw_prev_rents)

display({
    'contracts_shape': contracts.shape,
    'units_shape': units.shape,
    'prev_rents_shape': prev_rents.shape
})

# Peek
display(contracts.head(3))
display(units.head(3))
display(prev_rents.head(3))



### Insights (Initial Inspection)
- Note column name standardization (lowercase + underscores).  
- Confirm presence (or absence) of key fields: `amount_revenue`, `contract_status`, `property_name`, `contract_start_date`, `contract_end_date`, `previous_tenancy_rent`, `building_name`, `unit_type`, `rental_status`.  
- If keys differ (e.g., `property` vs. `property_name`), they will be aligned in the cleaning step.



## 3) Cleaning & Standardization

**What & Why:**  
- Harmonize critical field names across files.  
- Convert text to numeric (`amount_revenue`, `previous_tenancy_rent`).  
- Parse dates and coerce invalid values to `NaT`.  
- Trim and normalize string labels (e.g., `contract_status`, `rental_status`).  
- Keep a *data dictionary snapshot* after cleaning.


In [ ]:

# Create flexible column mapping to *target* names expected downstream
target_map_variants = {
    'amount_revenue': ['amount_(revenue)', 'amount_revenue', 'revenue', 'amount', 'rent_amount'],
    'contract_status': ['contract_status', 'status'],
    'property_name': ['property_name', 'property', 'property_title'],
    'contract_start_date': ['contract_start_date', 'start_date', 'contract_start'],
    'contract_end_date': ['contract_end_date', 'end_date', 'contract_end'],
    'previous_tenancy_rent': ['previous_tenancy_rent', 'previous_rent', 'prev_rent'],
    'building_name': ['building_name', 'building', 'property_name'],
    'unit_type': ['unit_type', 'type', 'unit_category'],
    'rental_status': ['rental_status', 'occupancy', 'unit_status', 'status']
}

def align_columns(df, mapping):
    df = df.copy()
    for tgt, variants in mapping.items():
        for v in variants:
            if v in df.columns and tgt not in df.columns:
                df = df.rename(columns={v: tgt})
                break
    return df

# Align columns
contracts = align_columns(contracts, target_map_variants)
units = align_columns(units, target_map_variants)
prev_rents = align_columns(prev_rents, target_map_variants)

# Parse numerics
for col in ['amount_revenue', 'previous_tenancy_rent']:
    for df in [contracts, units, prev_rents]:
        if col in df.columns:
            df[col] = df[col].apply(to_num)

# Parse dates
for col in ['contract_start_date', 'contract_end_date']:
    if col in contracts.columns:
        contracts[col] = contracts[col].apply(to_dt)

# Normalize strings
for col in ['contract_status', 'property_name', 'building_name', 'unit_type', 'rental_status']:
    for df in [contracts, units, prev_rents]:
        if col in df.columns:
            df[col] = df[col].astype('string').str.strip().str.replace('\s+', ' ', regex=True)

# Snapshot of schemas
def schema(df):
    return pd.DataFrame({'dtype': df.dtypes.astype(str), 'non_null': df.notna().sum(), 'nulls': df.isna().sum()})

display({'contracts_schema': schema(contracts).sort_index(),
         'units_schema': schema(units).sort_index(),
         'prev_rents_schema': schema(prev_rents).sort_index()})



### Insights (Cleaning)
- Numeric and date parsing completed with coercion rules.  
- Check remaining nulls — especially in join keys like `building_name` and `property_name`.  
- If `amount_revenue` is missing in contracts, confirm where revenue lives (e.g., unit-level vs. contract-level).  
- If `rental_status` is not present in units, derive occupancy from alternative columns (see next section).



## 4) Feature Engineering

**What & Why:**  
- Create *Expired* contract flag.  
- Derive occupancy flags from `rental_status`.  
- Create standardized building and property keys (uppercased/trimmed) to improve joins.  
- Build a unified grain for analysis (by `building_name`, `property_name`, and, if available, `unit_id`).


In [ ]:

# Expired contracts
if 'contract_end_date' in contracts.columns:
    contracts['is_expired'] = contracts['contract_end_date'].lt(TODAY)

# Occupancy from rental_status (heuristic: 'Occupied' vs others). Adjust mapping as needed.
def to_occupied(x):
    s = str(x).strip().lower()
    if s in ['occupied', 'leased', 'active', 'rented']:
        return 1
    if s in ['vacant', 'available', 'inactive', 'terminated']:
        return 0
    return np.nan

if 'rental_status' in units.columns:
    units['occupied_flag'] = units['rental_status'].apply(to_occupied)

# Canonical keys for joining
for df in [contracts, units, prev_rents]:
    if 'building_name' in df.columns:
        df['building_key'] = df['building_name'].astype('string').str.upper().str.strip()
    if 'property_name' in df.columns:
        df['property_key'] = df['property_name'].astype('string').str.upper().str.strip()

# Attempt joins: previous rents to contracts (on property/building if direct contract id not available)
contracts_aug = contracts.copy()
if 'previous_tenancy_rent' not in contracts_aug.columns and 'previous_tenancy_rent' in prev_rents.columns:
    # Preference: join on property_key then building_key fallbacks
    join_keys = []
    if 'property_key' in contracts_aug.columns and 'property_key' in prev_rents.columns:
        join_keys.append('property_key')
    if 'building_key' in contracts_aug.columns and 'building_key' in prev_rents.columns:
        join_keys.append('building_key')
    if join_keys:
        contracts_aug = contracts_aug.merge(
            prev_rents[[*join_keys, 'previous_tenancy_rent']].drop_duplicates(subset=join_keys, keep='last'),
            on=join_keys, how='left'
        )

contracts_aug['revenue_minus_previous'] = np.nan
if 'amount_revenue' in contracts_aug.columns and 'previous_tenancy_rent' in contracts_aug.columns:
    contracts_aug['revenue_minus_previous'] = contracts_aug['amount_revenue'] - contracts_aug['previous_tenancy_rent']

display(contracts_aug.head(5))



### Insights (Feature Engineering)
- `is_expired` flags contracts that ended before **2025-10-31**.  
- `occupied_flag` derived from `rental_status` (verify mapping against business definitions).  
- `revenue_minus_previous` provides a quick delta for current vs previous tenancy rent at the contract grain.  
- Validate join quality (match rates) and revise keys if needed.



## 5) Descriptive Statistics & Quality Checks

**What & Why:**  
- Basic stats (count, mean, std, percentiles) for revenue and previous rent.  
- Completeness checks and cross-tabulations for statuses.  
- Top-level KPIs: total revenue, average revenue, occupancy rate, and count of expired contracts.


In [ ]:

# Numeric summaries
num_cols = [c for c in ['amount_revenue', 'previous_tenancy_rent', 'revenue_minus_previous'] if c in contracts_aug.columns]
desc_stats = contracts_aug[num_cols].describe().T if num_cols else pd.DataFrame()

# Status distributions
status_tables = {}
for col in ['contract_status', 'rental_status', 'unit_type', 'building_name', 'property_name']:
    if col in contracts_aug.columns:
        status_tables[col] = topn(contracts_aug[col])

# Top-level KPIs
total_revenue = contracts_aug['amount_revenue'].sum(min_count=1) if 'amount_revenue' in contracts_aug.columns else np.nan
avg_revenue = contracts_aug['amount_revenue'].mean() if 'amount_revenue' in contracts_aug.columns else np.nan
expired_count = contracts_aug['is_expired'].sum() if 'is_expired' in contracts_aug.columns else np.nan

# Occupancy rate from units table
occupancy_rate = np.nan
if 'occupied_flag' in units.columns:
    occupancy_rate = (units['occupied_flag'].mean() * 100.0).round(2)

display({'desc_stats': desc_stats,
         'total_revenue': total_revenue,
         'avg_revenue': avg_revenue,
         'expired_contracts_count': expired_count,
         'portfolio_occupancy_rate_%': occupancy_rate,
         'status_tables_samples': {k: v.head(10) for k, v in status_tables.items()}})



### Insights (Statistics & Quality)
- Review revenue distribution for skew/outliers; consider Winsorization if needed.  
- Check completeness of `previous_tenancy_rent` and join match rates.  
- Portfolio occupancy rate computed from `units` table (confirm source of truth).  
- Investigate any unusually high expired count (seasonality, policy changes, data lag).



## 6) Visualizations

**What & Why:**  
- Visual summaries communicate patterns quickly to stakeholders.  
- Use histograms/box plots for revenue distribution and outliers.  
- Compare revenue vs previous rent to spot uplift or decline.  
- Rank buildings by occupancy and expired contracts to prioritize action.



### 6.a Revenue Distribution
**Goal:** Understand central tendency, spread, and potential outliers.


In [ ]:

# Histogram of revenue
if 'amount_revenue' in contracts_aug.columns:
    plt.figure()
    plt.hist(contracts_aug['amount_revenue'].dropna(), bins=40)
    plt.title('Distribution of Amount (Revenue)')
    plt.xlabel('Amount (Revenue)')
    plt.ylabel('Count')
    plt.show()

# Box plot of revenue by unit type (if available)
if 'amount_revenue' in contracts_aug.columns and 'unit_type' in contracts_aug.columns:
    plt.figure()
    sns.boxplot(data=contracts_aug, x='unit_type', y='amount_revenue')
    plt.title('Amount (Revenue) by Unit Type')
    plt.xticks(rotation=45, ha='right')
    plt.tight_layout()
    plt.show()



**Insights (Revenue Distribution):**  
- Identify heavy tails/outliers; consider log-scale views for clarity.  
- Compare medians across `unit_type` to detect systematically higher/lower rent categories.



### 6.b Revenue vs. Previous Tenancy Rent (Per Contract / Property)

**Goal:** Compare current revenue against previous rent to quantify growth/decline.


In [ ]:

# Scatter / bar comparison at aggregated level
if {'amount_revenue', 'previous_tenancy_rent'}.issubset(contracts_aug.columns):
    agg_cols = ['building_key'] if 'building_key' in contracts_aug.columns else []
    if 'property_key' in contracts_aug.columns:
        agg_cols.append('property_key')
    if not agg_cols:
        agg_cols = ['contract_status'] if 'contract_status' in contracts_aug.columns else []

    comp = (contracts_aug
            .groupby(agg_cols, dropna=False)[['amount_revenue', 'previous_tenancy_rent']]
            .mean()
            .reset_index()
            .sort_values('amount_revenue', ascending=False)
            .head(20))

    display(comp.head(10))

    # Bar plot for top groups (average revenue vs previous rent)
    comp_melt = comp.melt(id_vars=agg_cols, value_vars=['amount_revenue','previous_tenancy_rent'],
                          var_name='metric', value_name='value')
    plt.figure()
    sns.barplot(data=comp_melt, x=agg_cols[-1] if agg_cols else 'metric', y='value', hue='metric')
    plt.title('Average Revenue vs Previous Tenancy Rent (Top Groups)')
    plt.xticks(rotation=45, ha='right')
    plt.tight_layout()
    plt.show()



**Insights (Revenue vs Previous):**  
- Highlight buildings/properties with the largest positive/negative deltas to guide pricing reviews.  
- Investigate drivers (renovations, unit mix changes, concessions) for notable variances.



### 6.c Occupancy Rate per Building

**Goal:** Rank buildings by occupancy to identify underperformers and capacity pockets.


In [ ]:

if {'building_key','occupied_flag'}.issubset(units.columns):
    occ = (units.groupby('building_key', dropna=False)['occupied_flag']
                .mean().mul(100).round(2)
                .reset_index(name='occupancy_rate_%')
                .sort_values('occupancy_rate_%', ascending=False))

    display(occ.head(15))

    # Bar chart (top 20)
    top_occ = occ.head(20)
    plt.figure()
    sns.barplot(data=top_occ, x='building_key', y='occupancy_rate_%')
    plt.title('Top-20 Buildings by Occupancy Rate (%)')
    plt.xticks(rotation=45, ha='right')
    plt.tight_layout()
    plt.show()



**Insights (Occupancy):**  
- Flag buildings below target occupancy thresholds for tactical leasing actions.  
- Check seasonality or recent handovers influencing occupancy baselines.



### 6.d Expired Contracts per Building

**Goal:** Quantify contract expiries to prioritize renewals or new leasing activities.


In [ ]:

if {'building_key','is_expired'}.issubset(contracts_aug.columns):
    exp = (contracts_aug.groupby('building_key', dropna=False)['is_expired']
                    .sum()
                    .reset_index(name='expired_contracts_count')
                    .sort_values('expired_contracts_count', ascending=False))

    display(exp.head(15))

    # Bar chart (top 20)
    top_exp = exp.head(20)
    plt.figure()
    sns.barplot(data=top_exp, x='building_key', y='expired_contracts_count')
    plt.title('Top-20 Buildings by Expired Contracts (Count)')
    plt.xticks(rotation=45, ha='right')
    plt.tight_layout()
    plt.show()



**Insights (Expired Contracts):**  
- Buildings with many expiries may need proactive renewal campaigns.  
- Cross-check with occupancy trends to balance vacancy risk.



## 7) Findings & Next Steps (Section-by-Section)

- **Data Loading & Cleaning:** Summarize missing keys and any data quality flags (duplicates, odd zeros, extreme values).  
- **Feature Engineering:** Validate `is_expired` logic (policy cut-off) and `occupied_flag` mapping vs business glossary.  
- **Descriptive Stats:** Note skew/outliers; propose robust metrics (median, IQR).  
- **Visuals:** Identify top 3 buildings for each focus area (revenue uplift, low occupancy, many expiries) and propose actions (pricing review, campaigns, maintenance checks).



## 8) Executive Summary (for Non-Technical Stakeholders)

**Date:** 2025-10-30

**What we did:** Cleaned, aligned, and profiled three core datasets to quantify portfolio performance across **revenue**, **occupancy**, and **contract expirations**.

**Headline Metrics (examples; will populate after run):**
- **Total Revenue (contracts)** — reported sum of `Amount (Revenue)`  
- **Portfolio Occupancy** — share of units marked *Occupied*  
- **Expired Contracts** — count of contracts past end date as of **2025-10-31**

**Key takeaways (to be refined after running with real data):**
1. Revenue distribution shows (e.g.) concentration in a few buildings; candidates for pricing review and yield management.  
2. Several buildings have **below-target occupancy**; recommend targeted marketing and agent incentives.  
3. **Upcoming/expired leases** cluster in specific buildings; prioritize renewal outreach to reduce churn.

**Next Steps:**  
- Validate join keys with operations (confirm canonical `Building Name` / `Property Name`).  
- Align occupancy source-of-truth (Units vs Contracts).  
- Enrich with unit area, bedroom count, and concessions to support pricing elasticity analysis.  
- Automate monthly refresh and dashboarding.
